In [1]:
# Imports
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from datetime import datetime, timezone, timedelta
from scipy.stats import ks_2samp, mannwhitneyu, cramervonmises_2samp, anderson_ksamp
import warnings

warnings.filterwarnings("ignore")
pd.options.mode.chained_assignment = None

In [64]:
minset = pd.read_pickle('data/laatste_meting/minfinal.pkl.gz')

In [3]:
minset.shape

(37844, 64)

In [65]:
minset = minset[(minset['dc_type_dossier'] == '2i') &
                            (~minset.index.get_level_values(0).str.contains('bijlage')) &
                            (~minset.index.get_level_values(0).str.contains('inventaris')) &
                            (~minset.index.get_level_values(0).str.contains('verzoek'))]
minset.shape

(104868, 70)

In [66]:
minset['foi_publishedDate_dt'] = pd.to_datetime(minset['foi_publishedDate'], format='%Y-%m-%d')

In [67]:
minset['foi_publishedDate'].isna().value_counts(), minset['foi_publishedDate_dt'].isna().value_counts()

(foi_publishedDate
 False    103417
 True       1451
 Name: count, dtype: int64,
 foi_publishedDate_dt
 False    103417
 True       1451
 Name: count, dtype: int64)

## Filteren op datum

**2023:**  
Besluiten gepubliceerd op 1 januari 2023 tot en met 10 januari 2024, met een besluitdatum van 1 januari 2023, tot en met 31 december 2023.

**Matglas:**  
Besluiten gepubliceerd op 10 januari 2023 of eerder, met een beslisdatum van 1 januari 2022, tot en met 31 december 2022

**Ondraaglijk Traag:**  
In deze notebook heb ik de besluiten gepubliceerd in de periode oktober 2020 - september 2021 aangehouden, en voor het VWS besluiten gepubliceerd in de periode april 2019 - maart 2020.  

In het rapport Ondraaglijk Traag kon ik niet vinden welke periode voor welk ministerie was aangehouden.  

Dit komt uit sectie 1.3 van het rapport Ondraaglijk Traag:  
`Dit onderzoek is uitgevoerd in de tweede helft van 2021. De kwantitatieve analyse van Wob-verzoeken is gemaakt
op basis van de op Rijksoverheid.nl gepubliceerde Wob-verzoeken. Het betreft cijfers van de Wob-verzoeken die
gepubliceerd zijn in de periode oktober 2020 – september 2021 (NB: bij sommige departementen is een iets
andere periode gehanteerd, bijvoorbeeld november 2020 – oktober 2021). Alleen bij het ministerie van VWS is
een afwijkende onderzoeksperiode gekozen (april 2019 – maart 2020) omdat dit voor VWS het laatste ‘normale
jaar’ was waarin nog geen sprake was van gepubliceerde corona-gerelateerde Wob-verzoeken.`

In [68]:
min2023 = minset.loc[(minset['foi_decisionDate'] <= '2023-12-31') & 
                     (minset['foi_decisionDate'] >= '2023-01-01') & 
                     (minset['foi_publishedDate'] <= '2024-01-10') & 
                     (minset['foi_publishedDate'] >= '2023-01-01') &
                     (minset.index.get_level_values(1) == 1)]


min2022EX = minset[(minset['foi_decisionDate'] >= '2022-01-01') & 
                   (minset['foi_decisionDate'] <= '2022-12-31') &
                   (minset['foi_publishedDate_dt'] <= '2023-01-10')]


min2021VWS = minset[(minset['dc_publisher_name'] == 'Ministerie Van Volksgezondheid, Welzijn En Sport') &
                    (minset['foi_publishedDate_dt'] >= '2019-04-01') &
                    (minset['foi_publishedDate_dt'] <= '2020-03-31')]

min2021NONVWS = minset[(minset['foi_publishedDate_dt'] >= '2020-10-01') & 
                       (minset['foi_publishedDate_dt'] <= '2021-09-30') &
                       (minset['dc_publisher_name'] != 'Ministerie Van Volksgezondheid, Welzijn En Sport')]

min2021EX = pd.concat([min2021NONVWS, min2021VWS])

In [69]:
display(min2023['foi_dossierType'].isna().value_counts())
min2023[min2023.index.get_level_values(1) == 1].groupby('dc_publisher_name')['foi_dossierType'].count()

foi_dossierType
False    1467
Name: count, dtype: int64

dc_publisher_name
Ministerie Van Algemene Zaken                                54
Ministerie Van Binnenlandse Zaken En Koninkrijksrelaties     88
Ministerie Van Buitenlandse Zaken                            60
Ministerie Van Defensie                                      54
Ministerie Van Economische Zaken En Klimaat                 116
Ministerie Van Financiën                                    209
Ministerie Van Infrastructuur En Waterstaat                 183
Ministerie Van Justitie En Veiligheid                       171
Ministerie Van Landbouw, Natuur En Voedselkwaliteit         330
Ministerie Van Onderwijs, Cultuur En Wetenschap              89
Ministerie Van Sociale Zaken En Werkgelegenheid              40
Ministerie Van Volksgezondheid, Welzijn En Sport             73
Name: foi_dossierType, dtype: int64

# Aanpassingen

In de excelbestanden van Matglas stonden een paar doorlooptijden in de kolommen  
`Indien deelbesluit 1, aantal dagen` en `Indien deelbesluit 2, aantal dagen`.  
Als deze doorlooptijden worden gebruikt is het gemiddelde niet meer hetzelfde als in het Matglas rapport, deze doorlooptijden zijn dus niet meegerekend.  
Onderin wordt een vergelijking gemaakt waar deze wel worden meegerekend.  

In [70]:
min2021OT = pd.read_pickle('data/ExcelToDF/sheetsOT.pkl.gz', compression='gzip')
min2022MG = pd.read_pickle('data/ExcelToDF/sheetsMG.pkl.gz', compression='gzip')
min2022MG2 = pd.read_pickle('data/ExcelToDF/sheetsMG2.pkl.gz', compression='gzip')

min2021OT.rename(columns={"Aantal dagen \nin behandeling": "looptijd_foi", "Ministerie": "dc_publisher_name"}, inplace=True)
min2022MG.rename(columns={"Aantal dagen \nin behandeling": "looptijd_foi", "Ministerie": "dc_publisher_name"}, inplace=True)
min2022MG2.rename(columns={"Aantal dagen \nin behandeling": "looptijd_foi", "Ministerie": "dc_publisher_name"}, inplace=True)

In [71]:
def verschil_foi(row):
    dagtekening = row['foi_decisionDate']
    verzoek = row['verzoek_datum']
    if isinstance(dagtekening, datetime) and isinstance(verzoek, datetime):
        return dagtekening - verzoek
    else:
        return None
        
def td_to_days(td):
    if td != None:
        return td.days

## Outliers
Per ministerie wordt berekend wat het 95.5e percentiel is van alle doorlooptijden boven of gelijk aan 0.  
Alles wat hoger is dan dit limiet, wordt weggehaald.  
Ook worden negatieve doorlooptijden weggehaald. 

In [72]:
def remove_outliers_foi(df):
    limits = df[df['looptijd_foi'] >= 0].groupby('dc_publisher_name')['looptijd_foi'].quantile(0.995)
    loose_frames = []
    for ministerie in df['dc_publisher_name'].unique(): 
        min_filter = df[df['dc_publisher_name'] == ministerie]['looptijd_foi'] <= limits[ministerie]
        loose_frames.append(min_filter)

    cutoff_filter = pd.concat(loose_frames)
    df = df[(cutoff_filter) & (df['looptijd_foi'] >= 0)]
    return df

In [73]:
min2023['looptijd_foi'] = None
min2023['looptijd_foi'] = min2023.apply(lambda x: td_to_days(verschil_foi(x)), axis=1)
min2023['year'] = '2023'

min2022EX['looptijd_foi'] = None
min2022EX['looptijd_foi'] = min2022EX.apply(lambda x: td_to_days(verschil_foi(x)), axis=1)
min2022EX['year'] = '2022'

min2021EX['looptijd_foi'] = None
min2021EX['looptijd_foi'] = min2021EX.apply(lambda x: td_to_days(verschil_foi(x)), axis=1)
min2021EX['year'] = '2021'


min2022MG['year'] = '2022'
min2021OT['year'] = '2021'

# voor elk ministerie apart de extreme outliers weghalen (boven het 99,5e percentiel)
# als dit gebeurt voor de hele groep, blijven outliers van beter presterende ministeries staan
# zo hoop ik elk ministerie eerlijker te behandelen, aangezien ze ook apart worden vergeleken.

print('Niet gefiltered op uitschieters')
display(min2023.groupby('dc_publisher_name')['looptijd_foi'].describe())
print('')
print('Alles onder de 0 weg, en per ministerie alles boven het 99,5e percentiel')
display(remove_outliers_foi(min2023).groupby('dc_publisher_name')['looptijd_foi'].describe())

Niet gefiltered op uitschieters


,count,mean,std,min,25%,50%,75%,max
dc_publisher_name,,,,,,,,
Ministerie Van Algemene Zaken,54.0,147.685185,160.979880,0.0,38.25,102.5,176.00,901.0
Ministerie Van Binnenlandse Zaken En Koninkrijksrelaties,86.0,158.860465,213.388194,-247.0,52.00,105.5,178.00,1493.0
Ministerie Van Buitenlandse Zaken,59.0,128.915254,105.530236,9.0,47.00,113.0,178.00,596.0
Ministerie Van Defensie,48.0,124.208333,126.973101,8.0,33.75,69.0,166.75,551.0
Ministerie Van Economische Zaken En Klimaat,112.0,148.071429,128.735479,0.0,57.75,106.0,176.00,660.0
Ministerie Van Financiën,204.0,260.240196,356.730206,-79.0,107.25,184.5,314.25,3270.0
Ministerie Van Infrastructuur En Waterstaat,166.0,217.578313,259.606188,3.0,56.00,124.5,271.50,1222.0
Ministerie Van Justitie En Veiligheid,160.0,230.893750,293.505823,-5.0,85.50,139.5,260.25,2197.0
"Ministerie Van Landbouw, Natuur En Voedselkwaliteit",316.0,176.651899,285.590603,-1641.0,50.75,101.0,210.25,3372.0



Alles onder de 0 weg, en per ministerie alles boven het 99,5e percentiel


,count,mean,std,min,25%,50%,75%,max
dc_publisher_name,,,,,,,,
Ministerie Van Algemene Zaken,53.0,133.471698,123.668229,0.0,36.00,101.0,167.00,525.0
Ministerie Van Binnenlandse Zaken En Koninkrijksrelaties,84.0,147.809524,151.916470,0.0,52.00,105.5,172.00,784.0
Ministerie Van Buitenlandse Zaken,58.0,120.862069,86.246779,9.0,46.00,109.5,171.00,465.0
Ministerie Van Defensie,47.0,115.127660,111.483595,8.0,33.50,67.0,158.50,457.0
Ministerie Van Economische Zaken En Klimaat,111.0,143.459459,119.664363,0.0,57.50,105.0,173.50,560.0
Ministerie Van Financiën,201.0,232.338308,198.398652,5.0,108.00,184.0,300.00,1535.0
Ministerie Van Infrastructuur En Waterstaat,165.0,211.490909,248.228593,3.0,56.00,124.0,270.00,1218.0
Ministerie Van Justitie En Veiligheid,158.0,219.943038,249.291921,0.0,87.25,139.5,256.25,1496.0
"Ministerie Van Landbouw, Natuur En Voedselkwaliteit",313.0,169.172524,189.739790,0.0,51.00,101.0,207.00,1044.0


## Statistiek

Voor de dataframes die geautomatiseerde data hebben, worden de outliers weggehaald, en de negatieve doorlooptijden.  
Daarna worden de twee lijsten met doorlooptijden statistisch met elkaar getest, met de volgende statistieken:  
1. Cramer von Mises 2 sample test
2. Mann-Whitney U test
3. Kolmogorov-Smirnov test
4. Anderson-Darling test

In [74]:
def significantie(ministerie, df_extr, df_excel, print_output=True):
    
    df_extr = df_extr[df_extr.index.get_level_values(1) == 1]
    
    cutoff = df_extr[df_extr['dc_publisher_name'] == ministerie]['looptijd_foi'].quantile(.995)

    # looptijden weghalen die hoger zijn dan het maximale in de handmatige excels, en niet lager dan 0 zijn
    lt_excel = df_excel[(df_excel['looptijd_foi'] <= df_excel['looptijd_foi'].max()) &
                          (df_excel['dc_publisher_name'] == ministerie)]['looptijd_foi']

    lt_ext = df_extr.loc[(df_extr['looptijd_foi'] <= cutoff) & 
                                     (df_extr['looptijd_foi'] >= 0) &
                                     (df_extr['dc_publisher_name'] == ministerie)]['looptijd_foi']

    vs_df = pd.DataFrame([lt_excel.rename('Handmatig'), lt_ext.rename('Automatisch')]).T
    
    cvm2s = cramervonmises_2samp(lt_excel, lt_ext)
    mwu = mannwhitneyu(lt_excel, lt_ext, method="auto")
    ks2s = ks_2samp(lt_ext, lt_excel)
    adks = anderson_ksamp([lt_excel, lt_ext])
    
    len_excel = len(lt_excel) 
    len_extr = len(lt_ext)
    
    if print_output == True:
        print(ministerie)
        display(vs_df.describe())

        print(f"Cramer von Mises 2 sample   pvalue: {cvm2s.pvalue}, statistic: {cvm2s.statistic}")
        print(f"Mann-Whitney U              pvalue: {mwu.pvalue}, statistic: {mwu.statistic}")
        print(f"Kolmogorov-Smirnov 2 sample pvalue: {ks2s.pvalue}, statistic: {ks2s.statistic}")
        print(f"Anderson-Darling k sample   pvalue: {adks.pvalue}, statistic: {adks.statistic}")

        print('')
        print("====================================================================")
        print('')

    return cvm2s, mwu, ks2s, adks, len_extr, len_excel

In [75]:
def significantie_all(df_extr, df_excel):    
    
    # outliers weghalen in de handmatige excels, en doorlooptijden die niet lager dan 0 zijn
    df_extr = df_extr[df_extr.index.get_level_values(1) == 1]
    df_extr = remove_outliers_foi(df_extr)
    
    lt_excel = df_excel['looptijd_foi']
    lt_ext = df_extr['looptijd_foi']
    
    lt_excel.dropna(inplace=True)
    
    vs_df = pd.DataFrame([lt_excel.values, lt_ext.values]).T
    vs_df = vs_df.rename(columns={0: 'Handmatig', 1:'Automatisch'})
    display(vs_df.describe())
    
    cvm2s = cramervonmises_2samp(lt_excel, lt_ext)
    mwu = mannwhitneyu(lt_excel, lt_ext, method="auto")
    ks2s = ks_2samp(lt_excel, lt_ext)
    adks = anderson_ksamp([lt_excel, lt_ext])
    
    print(f"Cramer von Mises 2 sample   pvalue: {cvm2s.pvalue}, statistic: {cvm2s.statistic}")
    print(f"Mann-Whitney U              pvalue: {mwu.pvalue}, statistic: {mwu.statistic}")
    print(f"Kolmogorov-Smirnov 2 sample pvalue: {ks2s.pvalue}, statistic: {ks2s.statistic}")
    print(f"Anderson-Darling k sample   pvalue: {adks.pvalue}, statistic: {adks.statistic}")
    
    print('')
    print("====================================================================")
    print('')

In [76]:
def result_df(df_extr, df_excel, print_output=False):
    result_dict = {}
    for frame in df_extr.groupby('dc_publisher_name'):
        cvm2s, mwu, ks2s, adks, len_extr, len_excel = significantie(frame[0], frame[1], df_excel, print_output)

        results = {'Cramer von Mises 2 sample p-value':cvm2s.pvalue, 
               'Mann-Whitney U p-value':mwu.pvalue, 
               'Kolmogorov-Smirnov 2 sample p-value':ks2s.pvalue, 
               'Anderson-Darling k sample p-value':adks.pvalue,
               'N (Automatic method)':len_extr,
               'N (Matglas)':len_excel}
        
        result_dict[frame[0]] = results
        
    return pd.DataFrame.from_dict(result_dict, orient='index')

def result_df_stat(df_extr, df_excel, print_output=False):
    result_dict = {}
    for frame in df_extr.groupby('dc_publisher_name'):
        cvm2s, mwu, ks2s, adks, len_extr, len_excel = significantie(frame[0], frame[1], df_excel, print_output)

        results = {'Cramer von Mises 2 sample statistic':cvm2s.statistic, 
               'Mann-Whitney U statistic':mwu.statistic, 
               'Kolmogorov-Smirnov 2 sample statistic':ks2s.statistic, 
               'Anderson-Darling k sample statistic':adks.statistic,
               'N (Automatic method)':len_extr,
               'N (Matglas)':len_excel}
        
        result_dict[frame[0]] = results
        
    return pd.DataFrame.from_dict(result_dict, orient='index')

### Ondraaglijk Traag

Alle ministeries hebben bij alle tests een p-waarde hoger dan 0.05.  
Hieruit kan worden geconcludeerd dat voor elk ministerie de doorlooptijden **niet** significant verschillen van de handmatige data uit de Excel bestanden. 

In [77]:
display(result_df(min2021EX, min2021OT))
display(result_df_stat(min2021EX, min2021OT))

# result_df(min2021EX, min2021OT).to_csv('data/ExcelToDF/min2021statistiek_pvalue.csv')
# result_df_stat(min2021EX, min2021OT).to_csv('data/ExcelToDF/min2021statistiek_statistic.csv')

# print(result_df(min2021EX, min2021OT).to_markdown())
# print(result_df_stat(min2021EX, min2021OT).to_markdown())

,Cramer von Mises 2 sample p-value,Mann-Whitney U p-value,Kolmogorov-Smirnov 2 sample p-value,Anderson-Darling k sample p-value,N (Automatic method),N (Matglas)
Ministerie Van Algemene Zaken,0.999829,0.859432,0.999949,0.250000,26,29
Ministerie Van Binnenlandse Zaken En Koninkrijksrelaties,0.605839,0.383447,0.811981,0.250000,73,76
Ministerie Van Buitenlandse Zaken,0.462607,0.315328,0.408354,0.250000,51,51
Ministerie Van Defensie,0.993097,0.986820,0.997929,0.250000,30,44
Ministerie Van Economische Zaken En Klimaat,0.585595,0.329575,0.882919,0.250000,91,97
Ministerie Van Financiën,0.910600,0.691776,0.981355,0.250000,102,115
Ministerie Van Infrastructuur En Waterstaat,0.685081,0.370220,0.867118,0.250000,91,84
Ministerie Van Justitie En Veiligheid,0.657466,0.508506,0.672842,0.250000,86,85
"Ministerie Van Landbouw, Natuur En Voedselkwaliteit",0.894644,0.799167,0.981572,0.250000,235,227
"Ministerie Van Onderwijs, Cultuur En Wetenschap",0.640397,0.438871,0.751711,0.250000,41,49


,Cramer von Mises 2 sample statistic,Mann-Whitney U statistic,Kolmogorov-Smirnov 2 sample statistic,Anderson-Darling k sample statistic,N (Automatic method),N (Matglas)
Ministerie Van Algemene Zaken,0.018194,366.0,0.074271,-1.101054,26,29
Ministerie Van Binnenlandse Zaken En Koninkrijksrelaties,0.097164,2544.0,0.098955,-0.669157,73,76
Ministerie Van Buitenlandse Zaken,0.130238,1150.0,0.176471,-0.227695,51,51
Ministerie Van Defensie,0.026454,658.0,0.083333,-1.093984,30,44
Ministerie Van Economische Zaken En Klimaat,0.100920,4049.5,0.080888,-0.624107,91,97
Ministerie Van Financiën,0.045221,5681.5,0.059676,-0.923422,102,115
Ministerie Van Infrastructuur En Waterstaat,0.082377,3521.5,0.086081,-0.603981,91,84
Ministerie Van Justitie En Veiligheid,0.087293,3869.5,0.104378,-0.047124,86,85
"Ministerie Van Landbouw, Natuur En Voedselkwaliteit",0.047422,26307.0,0.041466,-0.936517,235,227
"Ministerie Van Onderwijs, Cultuur En Wetenschap",0.091511,1100.5,0.133897,-0.019432,41,49


### Matglas

Elk ministerie behalve het Ministerie van BZK heeft op alle tests een p-waarde hoger dan 0.05.  
Hieruit kan worden geconcludeerd dat voor elk ministerie, behalve BZK, de doorlooptijden **niet** significant verschilllen van de handmatige data uit de Excel bestanden. 

In [82]:
display(result_df(min2022EX, min2022MG))
display(result_df_stat(min2022EX, min2022MG))

# result_df(min2022EX, min2022MG_all).to_csv('data/ExcelToDF/min2022statistiek_pvalue.csv')
# result_df_stat(min2022EX, min2022MG_all).to_csv('data/ExcelToDF/min2022statistiek_statistic.csv')

# print(result_df(min2022EX, min2022MG_all).to_markdown())
# print(result_df_stat(min2022EX, min2022MG_all).to_markdown())

,Cramer von Mises 2 sample p-value,Mann-Whitney U p-value,Kolmogorov-Smirnov 2 sample p-value,Anderson-Darling k sample p-value,N (Automatic method),N (Matglas)
Ministerie Van Algemene Zaken,0.996337,1.000000,0.952048,0.250000,6,8
Ministerie Van Binnenlandse Zaken En Koninkrijksrelaties,0.786794,0.511071,0.919635,0.250000,90,92
Ministerie Van Buitenlandse Zaken,0.030647,0.028869,0.045860,0.036569,72,70
Ministerie Van Defensie,0.321631,0.237795,0.391083,0.250000,56,62
Ministerie Van Economische Zaken En Klimaat,0.498983,0.296889,0.817041,0.250000,88,92
Ministerie Van Financiën,0.288060,0.170993,0.546035,0.250000,133,132
Ministerie Van Infrastructuur En Waterstaat,0.134242,0.132333,0.231456,0.136828,126,120
Ministerie Van Justitie En Veiligheid,0.915092,0.655121,0.963805,0.250000,133,109
"Ministerie Van Landbouw, Natuur En Voedselkwaliteit",0.406738,0.252302,0.760917,0.250000,251,270
"Ministerie Van Onderwijs, Cultuur En Wetenschap",0.958581,0.773742,0.988734,0.250000,46,59


,Cramer von Mises 2 sample statistic,Mann-Whitney U statistic,Kolmogorov-Smirnov 2 sample statistic,Anderson-Darling k sample statistic,N (Automatic method),N (Matglas)
Ministerie Van Algemene Zaken,0.031994,23.5,0.250000,-0.988072,6,8
Ministerie Van Binnenlandse Zaken En Koninkrijksrelaties,0.065525,3906.0,0.077295,-0.903191,90,92
Ministerie Van Buitenlandse Zaken,0.544974,1984.0,0.224206,2.308791,72,70
Ministerie Van Defensie,0.176376,1516.5,0.158986,-0.239765,56,62
Ministerie Van Economische Zaken En Klimaat,0.120187,3683.0,0.089921,-0.464948,88,92
Ministerie Van Financiën,0.190419,7923.5,0.094612,0.026281,133,132
Ministerie Van Infrastructuur En Waterstaat,0.301466,6720.0,0.128968,0.919074,126,120
Ministerie Van Justitie En Veiligheid,0.044317,7006.0,0.061254,-0.910507,133,109
"Ministerie Van Landbouw, Natuur En Voedselkwaliteit",0.144854,31919.0,0.057046,-0.093935,251,270
"Ministerie Van Onderwijs, Cultuur En Wetenschap",0.036894,1312.0,0.080324,-0.774908,46,59


## Alle ministeries bij elkaar

### Ondraaglijk Traag
Als de doorlooptijden van alle ministeries worden samengevoegd, is de p-waarde op elke test hoger dan 0.05.  
Hieruit kan worden geconcludeerd dat de doorlooptijden niet significant verschillen van de handmatige data uit de Excel bestanden.

### Matglas
De tests lijken niet goed uit te komen op de data van Matglas, de p-waarde is lager dan 0.05 bij alle tests.  
Bij de Excel bestanden van Matglas staan op een tweede Excel sheet doorlooptijden van bezwaren en deelbesluiten.  
Bij de automatische berekening van de doorlooptijden zijn bezwaren en deelbesluiten ook meegeteld.  
Na het toevoegen van de doorlooptijden van de bezwaren en deelbesluiten, komen alle tests wel boven de 0.05 uit, waaruit geconcludeerd kan worden dat de doorlooptijden niet significant verschillen.  

In [83]:
print('Ondraaglijk Traag')
significantie_all(min2021EX, min2021OT)

print('Matglas')
significantie_all(min2022EX, min2022MG)

Ondraaglijk Traag


,Handmatig,Automatisch
count,981.000000,916.000000
mean,161.568807,175.036026
std,157.162150,180.125262
min,0.000000,0.000000
25%,69.000000,69.000000
50%,112.000000,115.500000
75%,189.000000,218.250000
max,1067.000000,1558.000000


Cramer von Mises 2 sample   pvalue: 0.20322947150778203, statistic: 0.23897917898295873
Mann-Whitney U              pvalue: 0.28536000810519635, statistic: 436561.0
Kolmogorov-Smirnov 2 sample pvalue: 0.32141051887515953, statistic: 0.04340326464840707
Anderson-Darling k sample   pvalue: 0.13636805432115576, statistic: 0.9225258282717955


Matglas


,Handmatig,Automatisch
count,1077.000000,1085.000000
mean,166.962860,192.065438
std,162.807041,185.702273
min,3.000000,0.000000
25%,62.000000,70.000000
50%,115.000000,133.000000
75%,217.000000,247.000000
max,1223.000000,2205.000000


Cramer von Mises 2 sample   pvalue: 0.0006169038258012627, statistic: 1.2584911286749048
Mann-Whitney U              pvalue: 0.0003433557088033909, statistic: 532313.0
Kolmogorov-Smirnov 2 sample pvalue: 0.009020914313379957, statistic: 0.07023007244051363
Anderson-Darling k sample   pvalue: 0.001, statistic: 7.647135832637487




In [84]:
min2022MG_Copy = min2022MG.copy()

min2022MG_Copy['looptijd_foi'] = min2022MG_Copy['looptijd_foi'].combine_first(min2022MG_Copy['Indien deelbesluit 1, aantal dagen']).combine_first(min2022MG_Copy['Indien deelbesluit 2, aantal dagen'])
min2022MG_all = pd.concat([min2022MG_Copy, min2022MG2])
min2022MG_all['looptijd_foi'] = min2022MG_all.apply(lambda x: pd.to_numeric(x['looptijd_foi'], errors='coerce'), axis=1)

significantie_all(min2022EX, min2022MG_all)

,Handmatig,Automatisch
count,1248.000000,1085.000000
mean,181.391026,192.065438
std,173.906793,185.702273
min,3.000000,0.000000
25%,64.000000,70.000000
50%,126.000000,133.000000
75%,240.250000,247.000000
max,1223.000000,2205.000000


Cramer von Mises 2 sample   pvalue: 0.2266341889826492, statistic: 0.22329684920998716
Mann-Whitney U              pvalue: 0.13271830760892137, statistic: 652640.5
Kolmogorov-Smirnov 2 sample pvalue: 0.5911499718906554, statistic: 0.031647317736027414
Anderson-Darling k sample   pvalue: 0.19009437711856178, statistic: 0.5838648234887065


